<a href="https://colab.research.google.com/github/kurakanja/113-2coding/blob/main/%E5%8F%B0%E9%90%B52024%E6%AF%8F%E7%AB%99%E9%80%B2%E5%87%BAPCA%E9%99%8D%E7%B6%AD%E8%88%87%E5%9C%96%E8%A1%A8%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2024台鐵每日進站各站人數
https://docs.google.com/spreadsheets/d/1e_NTRPGozJ_SxBv6Dh9iXsrnNPggotMgIPf24__A9gs/edit?usp=sharing

資料來源:https://data.gov.tw/dataset/8792

此colab連結:https://colab.research.google.com/drive/1AnHaVwbVtMJXcrji9Nu7k3AIgA5VhLTw?usp=sharing

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e_NTRPGozJ_SxBv6Dh9iXsrnNPggotMgIPf24__A9gs/edit?gid=958775479#gid=958775479')

In [3]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('每日各站進出站人數2024').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt
0,20240101,900,9613,10620
1,20240101,910,1254,1438
2,20240101,920,1685,1934
3,20240101,930,4718,5507
4,20240101,940,1737,2187


In [7]:
print(len(df))
print(df.columns)

86504
Index(['trnOpDate', 'staCode', 'gateInComingCnt', 'gateOutGoingCnt'], dtype='object')


In [10]:
df_expanded=df
df_expanded['Date'] = pd.to_datetime(df['trnOpDate']).dt.date
df_expanded['Weekday'] = pd.to_datetime(df['trnOpDate']).dt.day_name()
df_expanded.head()

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt,Date,Weekday
0,20240101,900,9613,10620,2024-01-01,Monday
1,20240101,910,1254,1438,2024-01-01,Monday
2,20240101,920,1685,1934,2024-01-01,Monday
3,20240101,930,4718,5507,2024-01-01,Monday
4,20240101,940,1737,2187,2024-01-01,Monday


In [15]:
theme = df_expanded['staCode'].unique()
theme

array(['900', '910', '920', '930', '940', '950', '960', '970', '980',
       '990', '1000', '1010', '1020', '1030', '1040', '1050', '1060',
       '1070', '1080', '1090', '1100', '1110', '1120', '1130', '1140',
       '1150', '1160', '1170', '1180', '1190', '1191', '1192', '1193',
       '1194', '1201', '1202', '1203', '1204', '1205', '1206', '1207',
       '1208', '1210', '1220', '1230', '1240', '1250', '2110', '2120',
       '2130', '2140', '2150', '2160', '2170', '2180', '2190', '2200',
       '2210', '2220', '2230', '2240', '2250', '2260', '3140', '3150',
       '3160', '3170', '3180', '3190', '3210', '3220', '3230', '3240',
       '3250', '3260', '3270', '3280', '3290', '3300', '3310', '3320',
       '3330', '3340', '3350', '3360', '3370', '3380', '3390', '3400',
       '3410', '3420', '3430', '3431', '3432', '3433', '3434', '3435',
       '3436', '3450', '3460', '3470', '3480', '3490', '4050', '4060',
       '4070', '4080', '4090', '4100', '4110', '4120', '4130', '4140',
       '

In [25]:
# 將 staCode 轉為數值（如果尚未是 int）
df_expanded['staCode'] = df_expanded['staCode'].astype(int)
df_expanded['gateInComingCnt'] = df_expanded['gateInComingCnt'].astype(int)
df_expanded['gateOutGoingCnt'] = df_expanded['gateOutGoingCnt'].astype(int)
# 根據 staCode 範圍新增分類欄位
def classify_line(code):
    if 900 <= code <= 1250:
        return '縱貫線（北段）'
    elif 1251 <= code <= 2260:
        return '海岸線（海線）'
    elif 3140 <= code <= 3350:
        return '臺中線（山線）'
    elif 3360 <= code <= 4400:
        return '縱貫線（南段）'
    elif 4400 <= code <= 5120:
        return '屏東線'
    elif 5130 <= code <= 6000:
        return '南迴線'
    elif 6010 <= code <= 7000:
        return '臺東線'
    elif 7010 <= code <= 7120:
        return '北迴線'
    elif 7130 <= code <= 7390:
        return '宜蘭線'
    else:
        return '其他路線'

df_expanded['line'] = df_expanded['staCode'].apply(classify_line)
df_expanded.head()

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt,Date,Weekday,line
0,20240101,900,9613,10620,2024-01-01,Monday,縱貫線（北段）
1,20240101,910,1254,1438,2024-01-01,Monday,縱貫線（北段）
2,20240101,920,1685,1934,2024-01-01,Monday,縱貫線（北段）
3,20240101,930,4718,5507,2024-01-01,Monday,縱貫線（北段）
4,20240101,940,1737,2187,2024-01-01,Monday,縱貫線（北段）


In [26]:
#增加進出比(進除以出，大於1代表大多從該站搭車離開，小於1代表人潮從該站湧入)
# 避免除以 0，先把 gateOutGoingCnt 為 0 的設為 NaN 或很小的數
df_expanded['gateOutGoingCnt'] = df_expanded['gateOutGoingCnt'].replace(0, 1e-6)

# 建立進出比欄位
df_expanded['in_out_ratio'] = df_expanded['gateInComingCnt'] / df_expanded['gateOutGoingCnt']
df_expanded.head()

,trnOpDate,staCode,gateInComingCnt,gateOutGoingCnt,Date,Weekday,line,in_out_ratio
0,20240101,900,9613,10620.0,2024-01-01,Monday,縱貫線（北段）,0.905179
1,20240101,910,1254,1438.0,2024-01-01,Monday,縱貫線（北段）,0.872045
2,20240101,920,1685,1934.0,2024-01-01,Monday,縱貫線（北段）,0.871251
3,20240101,930,4718,5507.0,2024-01-01,Monday,縱貫線（北段）,0.856728
4,20240101,940,1737,2187.0,2024-01-01,Monday,縱貫線（北段）,0.794239


寫回去確認數據(直接在程式中看df過大)

In [34]:
# 確保第二個 worksheet 存在（index 1）
worksheet2 = gsheets.get_worksheet(1)  # 第二頁 Sheet2（index 從 0 開始）

# 將 DataFrame 轉成 2D 陣列（含標題列）
data_to_write = [df_expanded.columns.tolist()] + df_expanded.values.tolist()

# 清空 Sheet2 再寫入（以防殘留資料）
worksheet2.clear()

# 如果你要一起寫欄位名稱
header = list(df_expanded.columns)
data_to_write = [header] + df_expanded.astype(str).values.tolist()

worksheet2.update(range_name='A1', values=data_to_write)

{'spreadsheetId': '1e_NTRPGozJ_SxBv6Dh9iXsrnNPggotMgIPf24__A9gs',
 'updatedRange': "'確認用'!A1:H86505",
 'updatedRows': 86505,
 'updatedColumns': 8,
 'updatedCells': 692040}

Grouping and Counting Behaviors per Theme

In [23]:
fig = px.bar(df_expanded, x='Date', y='gateInComingCnt', color='line',
             title="各路線每日進站人數",
             labels={'gateInComingCnt': '進站人數', 'line': '路線'},
             barmode='stack')
fig.show()

In [28]:
import plotly.graph_objects as go

# 先整理資料（選擇你要的欄位）
pivot_df = df_expanded.pivot_table(index='line', columns='Date', values='in_out_ratio', aggfunc='mean')

# 建立熱力圖
fig = go.Figure(data=go.Heatmap(
    z=pivot_df.values,
    x=pivot_df.columns.astype(str),  # 日期
    y=pivot_df.index.astype(str),    # line
    colorscale='Viridis',
    colorbar=dict(title='進出比'),
    zmin=0,  # 可視需求設最小最大值
    zmax=5
))

fig.update_layout(
    title='進出比熱力圖（line × 日期）',
    xaxis_title='日期',
    yaxis_title='車站代碼（line）',
    height=600
)

fig.show()

In [39]:
# 假設 df_expanded 中有 Date 欄位是 datetime 格式
df_expanded['month'] = df_expanded['Date'].dt.to_period('M').astype(str)  # e.g. '2025-02'

# 新增進出比分類欄位
df_expanded['ratio_group'] = df_expanded['in_out_ratio'].apply(
    lambda x: '進站 > 出站' if x > 1 else '進站 < 出站'
)

# 過濾有效資料
valid_df = df_expanded[
    df_expanded['in_out_ratio'].notna() &
    df_expanded['in_out_ratio'].apply(lambda x: x != float('inf'))
]

# 群組計算每條路線每個月各類型次數
monthly_line_stats = valid_df.groupby(['line', 'month', 'ratio_group']).size().reset_index(name='count')

In [44]:
import plotly.express as px

# 選擇某條路線
line_name = '縱貫線（北段）'
line_data = monthly_line_stats[monthly_line_stats['line'] == line_name]

fig = px.bar(line_data, x='month', y='count', color='ratio_group',
             barmode='group',
             title=f'{line_name} 每月進出比分類次數',
             labels={'month': '月份', 'count': '次數', 'ratio_group': '進出比類型'})

fig.show()

In [45]:
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

# 假設 df_expanded 已有下列欄位：
# ['staCode', 'line', 'gateInComingCnt', 'gateOutGoingCnt', 'Date']

# ➕ 建立進出比欄位（避免除以 0）
df_expanded = df_expanded.copy()
df_expanded['in_out_ratio'] = df_expanded['gateInComingCnt'] / df_expanded['gateOutGoingCnt'].replace(0, 1)

# ➕ 抽取時間欄位特徵（星期幾）
df_expanded['weekday'] = df_expanded['Date'].dt.weekday  # 0=週一，6=週日

# ➕ 編碼路線名稱為數字
le = LabelEncoder()
df_expanded['line_code'] = le.fit_transform(df_expanded['line'])

# 🔢 每個站點的行為摘要（你也可以用 groupby(['staCode', 'weekday']) 做更細分）
station_summary = df_expanded.groupby('staCode').agg({
    'gateInComingCnt': 'mean',
    'gateOutGoingCnt': 'mean',
    'in_out_ratio': 'mean',
    'weekday': 'mean',          # 看看是否有時間上的偏好
    'line_code': 'first'        # 每站只有一條 line，就取第一個
}).reset_index()

# 🚀 KMeans 聚類分析
features = ['gateInComingCnt', 'gateOutGoingCnt', 'in_out_ratio', 'weekday', 'line_code']
kmeans = KMeans(n_clusters=4, random_state=42)
station_summary['cluster'] = kmeans.fit_predict(station_summary[features])

# 🔽 PCA 降維做視覺化
pca = PCA(n_components=2)
components = pca.fit_transform(station_summary[features])
station_summary['pca1'] = components[:, 0]
station_summary['pca2'] = components[:, 1]

# 📊 繪圖
fig = px.scatter(
    station_summary,
    x='pca1',
    y='pca2',
    color='cluster',
    title="車站行為聚類結果（混合進出站特徵）",
    labels={'pca1': '主成分1', 'pca2': '主成分2', 'cluster': '群組'},
    hover_data=['staCode', 'gateInComingCnt', 'gateOutGoingCnt', 'in_out_ratio']
)

fig.update_layout(
    xaxis_title="主成分1",
    yaxis_title="主成分2",
    legend_title="聚類群組",
    title_x=0.5,
    height=600,
    width=800
)

fig.show()